<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_IOT_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Joseph Friedel          AAI530 IOT               Assignment 3                          ------- 27 Jan. 2025

Linear Regression Analysis and Prediction for IoT
This notebook holds the Assignment 3.1 for Module 3 in AAI 530, Data Analytics and the Internet of Things. In this assignment, you will use linear
regression to make predictions for simulated "streaming" data. The work that you do in this assignment will build on the linear regression predictions
that you saw in your text book and in this week's lab session. Be sure to answer the analysis questions thoroughly, as this is a large part of the
assignment for this week.

General Assignment Instructions
These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.
                                                                                          

One sign of mature code is conforming to a style guide. We recommend the Google Python Style Guide. If you use a different style guide, please include a
cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final
submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential import statements and make sure that
all such statements are moved into the designated cell.

When you save your notebook as a pdf, make sure that all cell output is visible (even error messages) as this will aid your instructor in grading your
work.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions
to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. Make sure to answer every question marked with a Q:
for full credit.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

#suppress scientific notation in pandas
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
#use this cell to import additional libraries or define helper functions
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error as mse

Load and prepare your data
We'll be using the cleaned household electric consumption dataset from Module 2 in this assignment. I recommend saving your dataset by running
df.to_csv("filename") at the end of the last assignment so that you don't have to re-do your cleaning steps. If you are not confident in your own
cleaning steps, you may ask your instructor for a cleaned version of the data. You will not be graded on the cleaning steps in this assignment, but some
functions may not work if you use the raw data.

We need to turn our datetime column into a numeric value to be used as a variable in our linear regression. In the lab session, we created a new column
of minutes and just incremented the value by 10 since we knew that the readings occurred every 10 minutes. In this dataset, we have readings every minute
, but we might have some missing rows depending on how you cleaned your data. So instead we will convert our datetime column to something called
unix/epoch time, which is the number of seconds since midnight on 1/1/1970.

TODO: load your data and convert the datetime column into epoch/unix time

In [ ]:
# Load the original data into 'df_raw'
df_raw = pd.read_csv("household_power_clean.csv")

# Clean the data and save it into 'df'
df = df_raw.copy()

# Remove rows with missing values
df.dropna(inplace=True)

# Reset the index after cleaning
df.reset_index(drop=True, inplace=True)

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(df.head())

In [ ]:
#convert datetime to epoch/unix time

# Ensure the 'Datetime' column is in the correct datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%m/%d/%Y %H:%M')

# Convert to Unix epoch time (seconds since Jan 1, 1970)
df['UnixTime'] = df['Datetime'].astype('int64') // 10**9

# Display the DataFrame to verify
print(df.head())

In [ ]:
print(df.tail(10))

Predicting Global Active Power
We will follow the code from the Chapter 9 in our textbook and the recorded lab session from this week to predict the Global Active Power (GAP) with
linear regression.

First we will create our x (time) and y (GAP) training variables, and then define our model parameters.

Q: What is ph? What is mu?

A:PH is the "points ahead".  It's how many data points ahead you want to predict.  The further ahead, the harder it is to do in a linear regression.  
Most likely your RMSE will increase as PH increases.

MU is the Decay Facter, telling you the importance of every suceeding point in your prediction.  A MU of "1" provides no decay, and your whole dataset
is used in the prediction.  If, for example mu=0.9, the 1st point is multiplied by 0.9^0, 2nd point 0.9^1, 3rd point 0.9^2, and so on.  So every
suceeding point is less important in the predication, making the more recent data more relevant.

TODO: Set the ph to be 5 minutes--consider the units that our time column is measured in.

In [ ]:
# Extracting Unix time and Global_active_power from the DataFrame
ts = pd.DataFrame(df['UnixTime'])
ys = pd.DataFrame(df['Global_active_power'])

# Set prediction horizon (ph) to 5 minutes in seconds
ph = 5 * 60  # 5 minutes converted to seconds

# Determine the data resolution (difference between consecutive timestamps)
time_resolution = ts.diff().dropna().iloc[0, 0]  # Assumes constant time intervals

# Calculate ph_index as the number of time steps in the prediction horizon
ph_index = int(ph / time_resolution)

# Define mu
mu = 0.9

#let's limit the number of samples in our model to 5000 just for speed
n_s = 5000

# Arrays to hold predicted values
tp_pred = np.zeros(n_s-1)
yp_pred = np.zeros(n_s-1)

print("ph (5 minutes in seconds):", ph)
print("Time resolution (seconds):", time_resolution)
print("ph_index (steps in 5 minutes):", ph_index)

In [ ]:
mu = 0.9
n_s = 5000

# Weight of the first data point on the last prediction
weight = mu**(n_s - 1)
print(f"Weight: {weight}")

Q: With mu = 0.9, how much weight will our first data point have on the last (5000th) prediction in our limited dataset?

A: Weight of the first data point on the 5000th prediction = mu^(n_s-1) ==0.9^4999 = ~0

TODO: Following the code from Chapter 9 and the lab session, use linear regression to predict a rolling GAP for our dataset. Store these predictions in
the tp_pred and yp_pred lists created above for visualization.

In [ ]:
# At every iteration of the for loop a new data sample is acquired
for i in range(2, n_s + 1):  # start out with 2 leading data points
    # Get x and y data "available" for our prediction
    ts_tmp=ts[0:i]
    ys_tmp=ys[0:i]
    ns = len(ys_tmp)

    # Initialize weights with exponential decay
    weights = np.ones(ns) * mu
    for k in range(ns):
        # Adjust weights to be downweighted according to their timestep away from our prediction
        weights[k]=weights[k]**k
    weights = np.flip(weights, 0)  # Flip to match order

    # Perform linear regression on "available" data using the mu-adjusted weights
    lm_tmp = LinearRegression()
    model_tmp=lm_tmp.fit(ts_tmp, ys_tmp, sample_weight=weights)

    # Store model coefficients (slope) and intercept to compute prediction
    m_tmp = model_tmp.coef_   # Slope
    q_tmp = model_tmp.intercept_  # Intercept

    # Use ph to make the model prediction according to the prediction time
    tp=ts.iloc[i-1,0]+ph
    yp = m_tmp * tp + q_tmp  # Linear model prediction for the prediction time

    # Store predictions for visualization
    tp_pred[i - 2] = tp
    yp_pred[i - 2] = yp

Now let's visualize the results from our model.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' %(mu, ph))
ax.plot(tp_pred, yp_pred, label='Predicted Value')
ax.plot(ts.iloc[0:n_s,0], ys.iloc[0:n_s,0], label='GAP data')
ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

It's difficult to tell how the model is performing from this plot.

TODO: Modify the code above to visualize the first and last 200 datapoints/predictions (can be in separate charts) and compute the MSE for our
predictions.

In [ ]:
#Plot first 200 data points/predictions
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the first 200 points of the predicted values
ax.plot(tp_pred[:200], yp_pred[:200], label='Predicted Value')

# Plot only the first 200 points of the actual data
ax.plot(ts.iloc[:200, 0], ys.iloc[:200, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
# Code to  plot a range of points
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Filter actual data within the specified time range
mask_actual = (ts.iloc[:, 0] >= 1166291820) & (ts.iloc[:, 0] <= 1166292820)
#mask_actual = (ts.iloc[:, 0] >=  1166291820) & (ts.iloc[:, 0] <= 1166292820)
ax.plot(ts.iloc[mask_actual.values, 0], ys.iloc[mask_actual.values, 0], label='GAP data')

# Filter predicted data within the same time range
mask_pred = (tp_pred >= 1166291820) & (tp_pred <= 1166292820)
#mask_pred = (tp_pred >=  1166291820) & (tp_pred <= 1166292820)
ax.plot(tp_pred[mask_pred], yp_pred[mask_pred], label='Predicted Value')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Plot last 200 data points/predictions
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the last 200 points of the predicted values
ax.plot(tp_pred[-200:], yp_pred[-200:], label='Predicted Value')

# Plot only the last 200 points of the actual data
ax.plot(ts.iloc[-200:, 0], ys.iloc[-200:, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Calculate MSE of predictions
print("MSE is", mse(ys['Global_active_power'][ph_index:5000+ph_index-1],yp_pred))

Q: How did our model perform? What do you observe on the charts? Is there a difference between the early and the late predictions? What does the MSE
tell you?

A: You can't tell look at all the data points.  When you look at small sections like 200 points, the predictions look poor.
Predictions are worse the end of the data.  In the full data plot, it looks like predictions are doing well or better in the middle of the data.
The MSE seem low, or good, at 0.58.  

TODO: Re-run the prediction code with mu = 1 and mu = 0.01. Use the cells below to produce charts for the first and last 200 points and to compute the
MSE for each of these sets of predictions.

In [ ]:
#Re-run prediction code for mu = 1
mu = 1

# At every iteration of the for loop a new data sample is acquired
for i in range(2, n_s + 1):  # start out with 2 leading data points
    # Get x and y data "available" for our prediction
    ts_tmp=ts[0:i]
    ys_tmp=ys[0:i]
    ns = len(ys_tmp)

    # Initialize weights with exponential decay
    weights = np.ones(ns) * mu
    for k in range(ns):
        # Adjust weights to be downweighted according to their timestep away from our prediction
        weights[k]=weights[k]**k
    weights = np.flip(weights, 0)  # Flip to match order

    # Perform linear regression on "available" data using the mu-adjusted weights
    lm_tmp = LinearRegression()
    model_tmp=lm_tmp.fit(ts_tmp, ys_tmp, sample_weight=weights)

    # Store model coefficients (slope) and intercept to compute prediction
    m_tmp = model_tmp.coef_   # Slope
    q_tmp = model_tmp.intercept_  # Intercept

    # Use ph to make the model prediction according to the prediction time
    tp=ts.iloc[i-1,0]+ph
    yp = m_tmp * tp + q_tmp  # Linear model prediction for the prediction time

    # Store predictions for visualization
    tp_pred[i - 2] = tp
    yp_pred[i - 2] = yp

In [ ]:
#Plot first 200 data points/predictions for mu = 1
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the first 200 points of the predicted values
ax.plot(tp_pred[:200], yp_pred[:200], label='Predicted Value')

# Plot only the first 200 points of the actual data
ax.plot(ts.iloc[:200, 0], ys.iloc[:200, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Plot last 200 data points/predictions for mu = 1
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the last 200 points of the predicted values
ax.plot(tp_pred[-200:], yp_pred[-200:], label='Predicted Value')

# Plot only the last 200 points of the actual data
ax.plot(ts.iloc[-200:, 0], ys.iloc[-200:, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Calculate MSE of predictions for mu = 1
print("MSE is", mse(ys['Global_active_power'][ph_index:5000+ph_index-1],yp_pred))

In [ ]:
#Re-run prediction code for mu = 0.01
mu = 0.01

# At every iteration of the for loop a new data sample is acquired
for i in range(2, n_s + 1):  # start out with 2 leading data points
    # Get x and y data "available" for our prediction
    ts_tmp=ts[0:i]
    ys_tmp=ys[0:i]
    ns = len(ys_tmp)

    # Initialize weights with exponential decay
    weights = np.ones(ns) * mu
    for k in range(ns):
        # Adjust weights to be downweighted according to their timestep away from our prediction
        weights[k]=weights[k]**k
    weights = np.flip(weights, 0)  # Flip to match order

    # Perform linear regression on "available" data using the mu-adjusted weights
    lm_tmp = LinearRegression()
    model_tmp=lm_tmp.fit(ts_tmp, ys_tmp, sample_weight=weights)

    # Store model coefficients (slope) and intercept to compute prediction
    m_tmp = model_tmp.coef_   # Slope
    q_tmp = model_tmp.intercept_  # Intercept

    # Use ph to make the model prediction according to the prediction time
    tp=ts.iloc[i-1,0]+ph
    yp = m_tmp * tp + q_tmp  # Linear model prediction for the prediction time

    # Store predictions for visualization
    tp_pred[i - 2] = tp
    yp_pred[i - 2] = yp

In [ ]:
#Plot first 200 data points/predictions for mu = 0.01
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the first 200 points of the predicted values
ax.plot(tp_pred[:200], yp_pred[:200], label='Predicted Value')

# Plot only the first 200 points of the actual data
ax.plot(ts.iloc[:200, 0], ys.iloc[:200, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Plot last 200 data points/predictions for mu = 0.01
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the last 200 points of the predicted values
ax.plot(tp_pred[-200:], yp_pred[-200:], label='Predicted Value')

# Plot only the last 200 points of the actual data
ax.plot(ts.iloc[-200:, 0], ys.iloc[-200:, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Calculate MSE of predictions for mu = 0.01
print("MSE is", mse(ys['Global_active_power'][ph_index:5000+ph_index-1],yp_pred))

Q: How did our mu = 1 model perform? What do you observe on the charts? Is there a difference between the early and the late predictions? What does the
MSE tell you?

A: It performed worse in graph and in MSE.
On the charts, less match between data and predicted.
For both mu=0.9 and mu=1, the prediction strikes me as poor.  Early plotting is working.  The late plotting looks the same and incorrect.   
I may have a coding error.
The MSE increased sslightly for mu=1, so although the graphs were similar, the MSE showed performance deteriorated going from mu = 0.9 to mu = 1.

Q: How did our mu = 0.01 model perform? What do you observe on the charts? Is there a difference between the early and the late predictions? What does
the MSE tell you?

A: For mu=0.01, the early points graph looked better.  The late points graph looked the same, possibly because of a coding error.
The MSE says the data is worse going to mu=0.01: from MSE=0.5 to 8!

Q: Which of these three models is the best? How do you know? Why does this make sense based on the mu parameter used?

A:  Based on MSE mu=0.9 is the best.  It makes sense in that you wouls expect to have a mu sweet spot, somewhere between 0 and 1.
But I think you'd have to more mu tests to find the optimum mu value.

Q: What could we do to improve our model and/or make it more realistic and useful?

A:  More data points, with a smaller delta between points (on the time axis).  Test more mu values to find yhe minimal MSE.  

TODO: Add voltage data as a second variable to our model and re-run the prediction code. Then visualize the first and last 200 points and compute the MSE

In [ ]:
#add voltage to the x-variables in our dataset

# Extracting Unix time and Global_active_power from the DataFrame
ts = pd.DataFrame(df['UnixTime'])
ys = pd.DataFrame(df['Voltage'])

# Set prediction horizon (ph) to 5 minutes in seconds
ph = 5 * 60  # 5 minutes converted to seconds

# Determine the data resolution (difference between consecutive timestamps)
time_resolution = ts.diff().dropna().iloc[0, 0]  # Assumes constant time intervals

# Calculate ph_index as the number of time steps in the prediction horizon
ph_index = int(ph / time_resolution)

# Define mu
mu = 0.9

#let's limit the number of samples in our model to 5000 just for speed
n_s = 5000


In [ ]:
#run the prediction code on your expanded dataset
#make sure to adjust your yp prediction to include the coefficients from time AND voltage
# Arrays to hold predicted values
tp_pred = np.zeros(n_s-1)
yp_pred = np.zeros(n_s-1)

print("ph (5 minutes in seconds):", ph)
print("Time resolution (seconds):", time_resolution)
print("ph_index (steps in 5 minutes):", ph_index)

In [ ]:
#Plot first 200 data points/predictions for the expanded dataset
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Voltage Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the first 200 points of the predicted values
ax.plot(tp_pred[:200], yp_pred[:200], label='Predicted Value')

# Plot only the first 200 points of the actual data
ax.plot(ts.iloc[:200, 0], ys.iloc[:200, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('volts')
ax.legend()

In [ ]:
#Plot last 200 data points/predictions for the expanded data
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Voltage Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the last 200 points of the predicted values
ax.plot(tp_pred[-200:], yp_pred[-200:], label='Predicted Value')

# Plot only the last 200 points of the actual data
ax.plot(ts.iloc[-200:, 0], ys.iloc[-200:, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('volts')
ax.legend()

In [ ]:
#Calculate MSE of predictions for the expanded data
print("MSE is", mse(ys['Voltage'][ph_index:5000+ph_index-1],yp_pred))

Q: How did the model performed when you added the voltage data? How does it compare to the models without it?

A: Horribly.
The model worked better for Power.

There are lots of other ways that we could try to improve our model while still using linear regression.

TODO: Choose one alternative model and re-run the prediction code. Some ideas include:

Use a moving average as the response variable
Make your prediction based on the time of day instead of as a continuous time series
Use a moving window to limit your predictions instead of using a mu factor

Q: Describe your alternative model and why it might improve your model

A: I used the Moving Average method.  Moving average may be better at capturing short-term trends, while LR looks at the whole dataset.
Moving average can be simplier to code, reducing the chaces of error.  Also moving averages, using averaging, may be better at noise reduction.

In [ ]:
#create your alternative training data here
# Extracting Unix time and Global_active_power from the DataFrame
ts = pd.DataFrame(df['UnixTime'])
ys = pd.DataFrame(df['Global_active_power'])

# Setting prediction horizon (ph) to 5 minutes in seconds
ph = 5 * 60  # 5 minutes converted to seconds

# Determine the data resolution (difference between consecutive timestamps)
time_resolution = ts.diff().dropna().iloc[0, 0]

# Calculate ph_index as the number of time steps in the prediction horizon
ph_index = int(ph / time_resolution)

# Define the window size for the moving average
window_size = 10

#limiting the number of samples in model to 5000 just for speed
n_s = 5000

In [ ]:
#re-run the prediction code here
# Arrays to hold predicted values
tp_pred = np.zeros(n_s-1)
yp_pred = np.zeros(n_s-1)

print("ph (5 minutes in seconds):", ph)
print("Time resolution (seconds):", time_resolution)
print("ph_index (steps in 5 minutes):", ph_index)

# At every iteration of the for loop, a new data sample is acquired
for i in range(window_size, n_s + 1):  # start out with window_size leading data points
    # Get available data for prediction
    ts_tmp = ts[0:i]
    ys_tmp = ys[0:i]

    # Calculate the moving average of the last 'window_size' observations
    moving_avg = ys_tmp.iloc[-window_size:].mean()  # Calculate average of last 'window_size' points

    # Store the prediction for this time step
    tp_pred[i - window_size] = ts.iloc[i - 1, 0] + ph  # Predicted time (in epoch) after the prediction horizon
    yp_pred[i - window_size] = moving_avg  # Prediction is the moving average of the last window_size values

In [ ]:
#Plot first 200 data points/predictions for alternative model
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the first 200 points of the predicted values
ax.plot(tp_pred[:200], yp_pred[:200], label='Predicted Value')

# Plot only the first 200 points of the actual data
ax.plot(ts.iloc[:200, 0], ys.iloc[:200, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Plot last 200 data points/predictions for alternative model
fig, ax = plt.subplots(figsize=(10, 10))
fig.suptitle('Global Active Power Prediction', fontsize=22, fontweight='bold')
ax.set_title('mu = %g, ph=%g ' % (mu, ph))

# Plot only the last 200 points of the predicted values
ax.plot(tp_pred[-200:], yp_pred[-200:], label='Predicted Value')

# Plot only the last 200 points of the actual data
ax.plot(ts.iloc[-200:, 0], ys.iloc[-200:, 0], label='GAP data')

ax.set_xlabel('time (epoch)')
ax.set_ylabel('kilowatts')
ax.legend()

In [ ]:
#Calculate MSE of predictions for alternative model
print("MSE is", mse(ys['Global_active_power'][ph_index:5000+ph_index-1],yp_pred))

Q: Did your alternative model improve on our previous results? What else could you do to improve the model while still using linear regression?

A: Yes.  The graphs looked similar but the MSE was cut by more than half: from 0.5 to 0.2.
Again, more data points over a longer time period, and reduce  the time step size.  Run simulations to find the optimum mu and ph.

It's worth noting that the results we're getting int his assignment are based on a pretty short predictive horizon of 5 minutes. If we were to increase
our predictive horizon, our results would likely be worse and there would be more room for optimizing and improving the predictions of our model.